---

Data Mining Assignment 03.

Due Date: 5 Ordibehesht 1395, before 10:00 AM.

Rename the file so that it contains the names of the members of the group.

Email the .ipynb file to "taherian.khu@gmail.com".

Do not forget to choose DM_Assignment03_names as email subject

---

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 3 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) 

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [13]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [14]:
def get_list_of_university_towns():
    text_file = open('university_towns.txt', 'r')
    lines = text_file.readlines()
    laststate='';ln=[];df = pd.DataFrame(columns =["State", "RegionName"])
    for l in lines:
        if '[edit]' in l:
            laststate=(l.split('[')[0]).strip()
        else:
            df.loc[len(df)]=[laststate,(l.split('(')[0]).strip()]
    return df

get_list_of_university_towns().head(6)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy


In [15]:
gdp=pd.read_excel('gdplev.xls', skiprows = 7, parse_cols=[4, 5, 6], names=['Quarter', 'GDP in billions of current dollars', 'GDP in billions of chained 2009 dollars'])
gdp = gdp.set_index('Quarter')
gdp

,GDP in billions of current dollars,GDP in billions of chained 2009 dollars
Quarter,,
1947q1,243.1,1934.5
1947q2,246.3,1932.3
1947q3,250.1,1930.3
1947q4,260.3,1960.7
1948q1,266.2,1989.5
1948q2,272.9,2021.9
1948q3,279.5,2033.2
1948q4,280.7,2035.3
1949q1,275.4,2007.5


In [16]:
def analyseRec(df):
    final_arr = []
    for col in df.columns:
        rec_start = []
        rec_end = []
        rec_bottom = []
        recession = False
        rec_min = (0, 0)
        for i in range(1, len(df[col]) - 1):
            if not recession and (df[col].values[i-1] > df[col].values[i] > df[col].values[i+1]):
                recession = True
                rec_min = (i+1, df[col].values[i+1])
                rec_start.append(df.index[i])
            elif recession and (df[col].values[i-1] < df[col].values[i] < df[col].values[i+1]):
                recession = False
                mini, minval = rec_min
                if (minval > df[col].values[i-1]):
                    rec_min = (i-1, df[col].values[i-1])
                mini, minval = rec_min
                rec_bottom.append(df.index[mini])
                rec_end.append(df.index[i])
            elif recession:
                mini, minval = rec_min
                if (minval > df[col].values[i]):
                    rec_min = (i, df[col].values[i])
        final_arr.append((rec_start, rec_end, rec_bottom))
    return final_arr

In [17]:
def get_recession_start():
#     '''Returns the year and quarter of the recession start time
    rec_start, rec_end, rec_bottom = analyseRec(gdp)[0]
    return rec_start

get_recession_start()

[u'1949q1', u'1953q3', u'1957q4', u'2008q4']

In [18]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    rec_start, rec_end, rec_bottom = analyseRec(gdp)[0]
    return rec_end

get_recession_end()

[u'1950q1', u'1954q2', u'1958q2', u'2009q3']

In [19]:
def get_recession_bottom():
    # recession bottom time 
    rec_start, rec_end, rec_bottom = analyseRec(gdp)[0]
    return rec_bottom

get_recession_bottom()

[u'1949q4', u'1954q1', u'1958q1', u'2009q2']

In [20]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    cols = [1, 2]
    cols.extend(range(10,251))
    global housing
    housing = pd.read_csv('City_Zhvi_AllHomes.csv', usecols=cols, index_col=[0, 1])
    for i in range(2000, 2017):
        for j in range(1, 12, 3):
            testcolname = str(i) + "-"
            if testcolname + str(j).zfill(2) in housing.columns:
                colname = str(i) + "q"
                workingarr = [testcolname + str(j).zfill(2)]
                if testcolname + str(j + 1).zfill(2) in housing.columns:
                    workingarr.append(testcolname + str(j + 1).zfill(2))
                if testcolname + str(j + 2).zfill(2) in housing.columns:
                    workingarr.append(testcolname + str(j + 2).zfill(2))
                housing[colname + str(int(j / 3) + 1)] = housing[workingarr].mean(axis= 1)
                housing.drop(labels= workingarr, axis=1, inplace=True)

    firstarr = ['1996-08', '1996-09']
    for i in range(1996, 2001):
        for j in range(1, 13):
            if str(i) + '-' + str(j).zfill(2) in housing.columns: 
                firstarr.append(str(i) + '-' + str(j).zfill(2))
    housing.drop(labels= firstarr, axis=1, inplace=True)
    return housing

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
RegionName,State,,,,,,,,,,,,,,,,,,,,,
New York,NY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.154667e+05,5.228000e+05,5.280667e+05,5.322667e+05,5.408000e+05,5.572000e+05,5.728333e+05,5.828667e+05,5.916333e+05,587200.0
Los Angeles,CA,2.070667e+05,2.144667e+05,2.209667e+05,2.261667e+05,2.330000e+05,2.391000e+05,2.450667e+05,2.530333e+05,2.619667e+05,2.727000e+05,...,4.980333e+05,5.090667e+05,5.188667e+05,5.288000e+05,5.381667e+05,5.472667e+05,5.577333e+05,5.660333e+05,5.774667e+05,584050.0
Chicago,IL,1.384000e+05,1.436333e+05,1.478667e+05,1.521333e+05,1.569333e+05,1.618000e+05,1.664000e+05,1.704333e+05,1.755000e+05,1.775667e+05,...,1.926333e+05,1.957667e+05,2.012667e+05,2.010667e+05,2.060333e+05,2.083000e+05,2.079000e+05,2.060667e+05,2.082000e+05,212000.0
Philadelphia,PA,5.300000e+04,5.363333e+04,5.413333e+04,5.470000e+04,5.533333e+04,5.553333e+04,5.626667e+04,5.753333e+04,5.913333e+04,6.073333e+04,...,1.137333e+05,1.153000e+05,1.156667e+05,1.162000e+05,1.179667e+05,1.212333e+05,1.222000e+05,1.234333e+05,1.269333e+05,128700.0
Phoenix,AZ,1.118333e+05,1.143667e+05,1.160000e+05,1.174000e+05,1.196000e+05,1.215667e+05,1.227000e+05,1.243000e+05,1.265333e+05,1.283667e+05,...,1.642667e+05,1.653667e+05,1.685000e+05,1.715333e+05,1.741667e+05,1.790667e+05,1.838333e+05,1.879000e+05,1.914333e+05,195200.0
Las Vegas,NV,1.326000e+05,1.343667e+05,1.354000e+05,1.370000e+05,1.395333e+05,1.417333e+05,1.433667e+05,1.461333e+05,1.493333e+05,1.509333e+05,...,1.700667e+05,1.734000e+05,1.754667e+05,1.775000e+05,1.816000e+05,1.867667e+05,1.906333e+05,1.946000e+05,1.972000e+05,199950.0
San Diego,CA,2.229000e+05,2.343667e+05,2.454333e+05,2.560333e+05,2.672000e+05,2.762667e+05,2.845000e+05,2.919333e+05,3.012333e+05,3.128667e+05,...,4.802000e+05,4.890333e+05,4.964333e+05,5.033667e+05,5.120667e+05,5.197667e+05,5.254667e+05,5.293333e+05,5.362333e+05,539750.0
Dallas,TX,8.446667e+04,8.386667e+04,8.486667e+04,8.783333e+04,8.973333e+04,8.930000e+04,8.906667e+04,9.090000e+04,9.256667e+04,9.380000e+04,...,1.066333e+05,1.089000e+05,1.115333e+05,1.137000e+05,1.211333e+05,1.285667e+05,1.346000e+05,1.405000e+05,1.446000e+05,149300.0
San Jose,CA,3.742667e+05,4.065667e+05,4.318667e+05,4.555000e+05,4.706667e+05,4.702000e+05,4.568000e+05,4.455667e+05,4.414333e+05,4.577667e+05,...,6.794000e+05,6.970333e+05,7.149333e+05,7.314333e+05,7.567333e+05,7.764000e+05,7.891333e+05,8.036000e+05,8.189333e+05,822200.0


In [22]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    Return the tuple (different, p, better) 
    where  
    different=True if the t-test is True at a p<0.01 (we reject the null hypothesis), 
    different=False if otherwise (we cannot reject the null hypothesis). 
    The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    data = convert_housing_data_to_quarters().copy()
    data = data.loc[:,'2008q3':'2009q2']
    data = data.reset_index()
    def price_ratio(row):
        return (row['2008q3'] - row['2009q2'])/row['2008q3']
    
    data['up&down'] = data.apply(price_ratio,axis=1)
    #uni data 
    
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)

    def is_uni_town(row):
        #check if the town is a university towns or not.
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    data['is_uni'] = data.apply(is_uni_town,axis=1)  
    not_uni = data[data['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni  = data[data['is_uni']==1].loc[:,'up&down'].dropna()
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    result = (True,p_val,better())
    return result
run_ttest()

(True, 0.00036641601595526971, 'university town')